In [6]:
import os
from eodag import setup_logging
setup_logging(verbose=2)
from eodag.api.core import EODataAccessGateway

In [2]:
dag = EODataAccessGateway()

STAC API compliant if provider have a "search".

Let's explore first of all static catalog

https://eodag.readthedocs.io/en/stable/notebooks/tutos/tuto_stac_client.html

In [27]:
# create a workspace
workspace = 'eodag_workspace'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

In [37]:
# add the provider
dag.update_providers_config("""
maxar_stac:
    search:
        type: StaticStacSearch
        api_endpoint: https://maxar-opendata.s3.amazonaws.com/events/catalog.json
        collections: Morocco-Earthquake-Sept-2023/collection.json
    products:
        GENERIC_PRODUCT_TYPE:
            productType: 'maxar_open_data'
    download:
        type: HTTPDownload
        base_uri: https://fake-endpoint
        flatten_top_dirs: True
        outputs_prefix: %s
""" % os.path.abspath(workspace))

In [38]:
dag.set_preferred_provider("maxar_stac")


In [39]:
from shapely.geometry import box

marakesh_bbox = [-8.140182,31.514167,-7.805099,31.727583]
search_bbox = box(*marakesh_bbox)
collection = "Morocco-Earthquake-Sept-2023/collection.json"
query_args = {"start": "2023-08-01", "end": "2023-10-06" , "geom": search_bbox, "provider": "maxar_stac", "collections":collection}

In [40]:
%%time
products, found = dag.search(**query_args)
print("%s product(s) found" % found)

2023-12-06 09:24:32,352 eodag.core                       [INFO    ] No product type could be guessed with provided arguments
2023-12-06 09:24:32,382 eodag.core                       [INFO    ] Searching product type 'None' on provider: maxar_stac
2023-12-06 09:38:29,692 eodag.core                       [INFO    ] No result from provider 'maxar_stac' due to an error during search. Raise verbosity of log messages for details
2023-12-06 09:38:29,694 eodag.core                       [ERROR   ] Error while searching on provider maxar_stac (ignored):
Traceback (most recent call last):
  File "/home/rustt/miniforge3/envs/eodag/lib/python3.10/site-packages/eodag/api/core.py", line 1540, in _do_search
    res, nb_res = search_plugin.query(count=count, auth=auth_plugin, **kwargs)
  File "/home/rustt/miniforge3/envs/eodag/lib/python3.10/site-packages/eodag/plugins/search/static_stac_search.py", line 74, in query
    features = fetch_stac_items(
  File "/home/rustt/miniforge3/envs/eodag/lib/python

0 product(s) found
CPU times: user 15.5 s, sys: 988 ms, total: 16.5 s
Wall time: 13min 57s


In [21]:
%%time
products, found = dag.search(**query_args)
print("%s product(s) found" % found)

2023-12-06 08:36:20,696 eodag.core                       [INFO    ] No product type could be guessed with provided arguments
2023-12-06 08:36:20,724 eodag.core                       [INFO    ] Searching product type 'None' on provider: maxar_stac
2023-12-06 08:50:33,128 eodag.core                       [INFO    ] No result from provider 'maxar_stac' due to an error during search. Raise verbosity of log messages for details
2023-12-06 08:50:33,130 eodag.core                       [ERROR   ] Error while searching on provider maxar_stac (ignored):
Traceback (most recent call last):
  File "/home/rustt/miniforge3/envs/eodag/lib/python3.10/site-packages/eodag/api/core.py", line 1540, in _do_search
    res, nb_res = search_plugin.query(count=count, auth=auth_plugin, **kwargs)
  File "/home/rustt/miniforge3/envs/eodag/lib/python3.10/site-packages/eodag/plugins/search/static_stac_search.py", line 74, in query
    features = fetch_stac_items(
  File "/home/rustt/miniforge3/envs/eodag/lib/python

4264 product(s) found
CPU times: user 23 s, sys: 1.71 s, total: 24.7 s
Wall time: 14min 26s


In [25]:
ids = [product.as_dict() for product in products]

In [26]:
ids[0]

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': (((-37.845, -53.8504),
    (-36.9164, -54.16),
    (-35.9968, -54.4557),
    (-35.0529, -54.7475),
    (-34.0954, -55.032),
    (-33.1245, -55.3087),
    (-32.14, -55.5776),
    (-31.1423, -55.8385),
    (-30.1314, -56.0912),
    (-29.1073, -56.3355),
    (-28.0702, -56.5711),
    (-27.0205, -56.7981),
    (-25.9583, -57.0161),
    (-24.8839, -57.225),
    (-23.7974, -57.4245),
    (-22.6994, -57.6146),
    (-21.5901, -57.795),
    (-20.4701, -57.9657),
    (-19.3396, -58.1263),
    (-18.1991, -58.277),
    (-17.6681, -57.0888),
    (-16.9254, -55.3175),
    (-16.23, -53.543),
    (-15.5749, -51.7665),
    (-14.9551, -49.9868),
    (-14.3662, -48.2045),
    (-13.8032, -46.4215),
    (-13.2637, -44.6351),
    (-12.7443, -42.8477),
    (-12.2433, -41.0575),
    (-11.7583, -39.2659),
    (-11.2864, -37.4737),
    (-10.8269, -35.6797),
    (-10.3787, -33.8836),
    (-9.93943, -32.0872),
    (-9.50903, -30.2891),
    (-9.

### Exemple documentation

In [12]:
# create a workspace
workspace = 'eodag_workspace_stac_client'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

# add the provider
dag.update_providers_config("""
stac_http_provider:
    search:
        type: StaticStacSearch
        api_endpoint: https://canada-spot-ortho.s3.amazonaws.com/canada_spot_orthoimages/canada_spot5_orthoimages/S5_2007/catalog.json
    products:
        GENERIC_PRODUCT_TYPE:
            productType: '{productType}'
    download:
        type: HTTPDownload
        base_uri: https://fake-endpoint
        flatten_top_dirs: True
        outputs_prefix: %s
""" % os.path.abspath(workspace))

dag.set_preferred_provider("stac_http_provider")

from shapely.geometry import Polygon

search_polygon = Polygon([(-70, 45), (-75, 47), (-80, 47), (-80, 44)])
query_args = {"start": "2007-05-01", "end": "2007-05-06" , "geom": search_polygon}

products, found = dag.search(**query_args)
print("%s product(s) found" % found)

2023-12-06 08:31:01,524 eodag.config                     [INFO    ] stac_http_provider: unknown provider found in user conf, trying to use provided configuration
2023-12-06 08:31:01,527 eodag.core                       [INFO    ] No product type could be guessed with provided arguments
2023-12-06 08:31:01,527 eodag.config                     [INFO    ] Fetching external product types from https://cs-si.github.io/eodag/eodag/resources/ext_product_types.json
2023-12-06 08:31:01,981 eodag.core                       [INFO    ] Searching product type 'None' on provider: stac_http_provider
2023-12-06 08:31:06,542 eodag.crunch.date                [INFO    ] Finished filtering products. 21 resulting products
2023-12-06 08:31:06,545 eodag.crunch.overlap             [INFO    ] Finished filtering products. 3 resulting products
2023-12-06 08:31:06,552 eodag.core                       [INFO    ] Found 3 result(s) on provider 'stac_http_provider'


3 product(s) found


In [13]:
# plot products and search polygon on map
import ipyleaflet as ipyl

m = ipyl.Map(center=(45, -75), zoom=5)

polygon_layer = ipyl.GeoJSON(data=search_polygon.__geo_interface__, style=dict(color='blue'))
m.add_layer(polygon_layer)

items_layer = ipyl.GeoJSON(data=products.as_geojson_object(), style=dict(color='green'))
m.add_layer(items_layer)

ModuleNotFoundError: No module named 'ipyleaflet'